[View in Colaboratory](https://colab.research.google.com/github/rkhaled98/Karmalutional-Network/blob/master/scripts/KARMALUTIONALNETWORKKERAS.ipynb)

First, download the glove word embedding of your choice:

50d version (smaller, use with colaboratory): https://www.kaggle.com/devjyotichandra/glove6b50dtxt

Make sure this file is named glove.6B.50d.txt <br /> <br />

300d version (larger but more accurate): http://nlp.stanford.edu/data/glove.42B.300d.zip

Make sure this file is named glove.42B.300d.txt <br /> <br />


Move the file to the 'Karmalutional-Network/data' directory (they were too big for github) and set the variable below to the dimension you chose


In [24]:
glove_dim = 50

Click above "View in Colaboratory" link to view in Google Colaboratory. If not using Colaboratory, skip next 4 cells 

In [81]:
from google.colab import files
uploaded = files.upload()
# get these files exactly from git repo under data folder: data/glove.6B.50d.txt,
# data/news_full_train.csv and data/news_full_test.csv


In [24]:
!mkdir data

datalab		    glove.6B.50d.txt	      news_news_small_test_1 (1).csv
glove.42B.300d.txt  news_news_small_1.csv     news_news_small_test_1.csv
glove.42B.300d.zip  news_news_small.outfinal


In [ ]:
!mv glove.6B.50d.txt news_full_train.csv news_full_test.csv data

In [10]:
!ls # make sure there are the files required above in the folder

datalab		    glove.6B.50d.txt	      news_news_small_test_1 (1).csv
glove.42B.300d.txt  news_news_small_1.csv     news_news_small_test_1.csv
glove.42B.300d.zip  news_news_small.outfinal


If not using Colaboratory, begin here

In [30]:
import csv
import numpy as np
from pathlib import Path


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
  
  
def read_csv(filename):
    phrase = []
    rank = []

    with open(filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[1])
            if row[2] == "":
                rank.append('0')
            else:
                rank.append(row[2])

    X = np.asarray(phrase)
    # print(X)
    Y = np.asarray(rank)
    return X, Y
  
  
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [31]:
def predict(X, Y, W, b, word_to_vec_map, output_prediction=False):
    """
    Given X (sentences) and Y (relative score value), predict the Y and get the accuracy.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing relative score index, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()
        
        # Average words' vectors
        avg = np.zeros((glove_dim,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
        if output_prediction:
           print("The model predicts that the comment %s will receive a relative score ranking of %s" % (X[j], pred[j]))
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred

In [32]:
def predict_one(X, W, b, word_to_vec_map):
    words = X.lower().split()
    avg = np.zeros((glove_dim,))
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg/len(words)
    Z = np.dot(W, avg) + b
    A = softmax(Z)
    pred = np.argmax(A)
    print("The model predicts that the comment: \n'%s'\n will receive a relative score ranking of: %s" % (X, pred))

In [9]:
path = Path.cwd()
print("reading training data")
X_train, Y_train = read_csv(path/'data/news_full_train.csv') # the data needed by the read_csv is a csv with comment,rank generated by the datasets main script 
print("done reading training data")
Y_train = Y_train.astype(int)

print("reading testing data")
X_test, Y_test = read_csv(path/'data/news_full_test.csv')
print("done reading testing data")

Y_test = Y_test.astype(int)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

reading training data


done reading training data
reading testing data


done reading testing data
(58377,) (58377,)
(14594,) (14594,)


In [10]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [23]:
print("reading glove vector")
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(Path.cwd()/'data/glove.6B.50d.txt')
print("done reading glove vector")

reading glove vector


done reading glove vector


In [25]:
def remove_non_keys(X, dct):
    for i in range(0, len(X)):
        for word in X[i].split(" "):
            if not (word in dct):
                if not (word == " "):
                    if not (word == ""):
                        X[i] = X[i].replace(word, '')
                        X[i] = " ".join(X[i].split())
    return X

In [26]:
print("removing non keys")
X_train = remove_non_keys(X_train, word_to_index)
X_train = remove_non_keys(X_train, word_to_index)
X_test = remove_non_keys(X_test, word_to_index)
X_test = remove_non_keys(X_test, word_to_index)
print("done removing non keys")

removing non keys


done removing non keys


In [33]:
def sentence_to_avg(sentence, word_to_vec_map):
  
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 300-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (300,)
    """
    
    #Split sentence into list of lower case words (around 1 line)
    words = [i.lower() for i in sentence.split()]

    # Initialize the average word vector
    avg = np.zeros((glove_dim,))
    
    #average the word vectors
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg / len(words)
    
    return avg

In [34]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    """
    Model to train word vector representations in numpy.
    
    Arguments:
    X -- input data, numpy array of comment sentences as strings, of shape (m, 1)
    Y -- labels, numpy array of integers between 0 and 9, numpy-array of shape (m, 1)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    learning_rate -- learning_rate for the stochastic gradient descent algorithm
    num_iterations -- number of iterations
    
    Returns:
    pred -- vector of predictions, numpy-array of shape (m, 1)
    W -- weight matrix of the softmax layer, of shape (n_y, n_h)
    b -- bias of the softmax layer, of shape (n_y,)
    """
    

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 10                                 # number of classes  
    n_h = glove_dim                               # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    alpha,hidden_dim,dropout_percent,do_dropout = (0.5,10,0.1,False) # playing around with dropout
     
    # Optimization loop
    for t in range(num_iterations):                       # Loop over the number of iterations
        for i in range(m):                                # Loop over the training examples
            
            # Average the word vectors of the words from the i'th training example
            avg = sentence_to_avg(X[i], word_to_vec_map)

            # Forward propagate the avg through the softmax layer
            z = np.dot(W, avg) + b
            dropout = np.random.binomial([np.ones((hidden_dim))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))
            if do_dropout:
                z *= dropout
            a = softmax(z)

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -np.sum(np.multiply(Y_oh[i], np.log(a)))
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 20 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            #W *= 
#             print(z)
#             print(dropout.shape)
#             print(a)
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

In [ ]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

Epoch: 0 --- cost = 2.89329016312415


ValueError: operands could not be broadcast together with shapes (300,) (50,) (300,) 

In [76]:
predict_one("Rafi is the best", W, b, word_to_vec_map)
predict_one("Nanak is the best", W, b, word_to_vec_map)
predict_one("Colby is the best", W, b, word_to_vec_map)
predict_one("Brian is the best", W, b, word_to_vec_map)
predict_one("Kevin is the best but Rafi is better", W, b, word_to_vec_map)
predict_one("Victor is the best", W, b, word_to_vec_map)
predict_one("warrior princess is the best", W, b, word_to_vec_map)
# predict_one("Victor is the greatest of all time", W, b, word_to_vec_map)

The model predicts that the comment: 
'Rafi is the best'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Nanak is the best'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Colby is the best'
 will receive a relative score ranking of: 1
The model predicts that the comment: 
'Brian is the best'
 will receive a relative score ranking of: 8
The model predicts that the comment: 
'Kevin is the best but Rafi is better'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Victor is the best'
 will receive a relative score ranking of: 5
The model predicts that the comment: 
'warrior princess is the best'
 will receive a relative score ranking of: 8


In [14]:
import numpy as np
from tensorflow import keras

In [15]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [w.lower() for w in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j += 1
            
    ### END CODE HERE ###
    
    return X_indices

In [16]:

X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[ 918419. 1184974.       0.       0.       0.]
 [1165414. 1435116.  554227.       0.       0.]
 [ 899172. 1071293. 1510241.  900259. 1900781.]]


In [17]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = keras.layers.Embedding(vocab_len, emb_dim, trainable=False)

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [18]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.066973


In [19]:
def Model_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Model-V2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    sentence_indices = keras.layers.Input(input_shape, dtype='int32')

    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)   
    
    X = keras.layers.LSTM(128, return_sequences=True)(embeddings)
    X = keras.layers.Dropout(0.5)(X)
    X = keras.layers.LSTM(128, return_sequences=False)(X)
    X = keras.layers.Dropout(0.5)(X)
    X = keras.layers.Dense(10)(X)
    X = keras.layers.Activation('softmax')(X)
    
    model = keras.Model(inputs=sentence_indices, outputs=X)
   
    
    return model

In [20]:
maxLen = len(max(X_train, key=len).split())

In [21]:
model = Model_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 509)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 509, 300)          575248800 
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
__________

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 10)

In [33]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 10, shuffle=True)

Epoch 1/10


  10/3852 [..............................]  10/3852 [..............................] - ETA: 13:05 - loss: 2.1268 - acc: 0.2000

  20/3852 [..............................]  20/3852 [..............................] - ETA: 11:36 - loss: 2.1026 - acc: 0.2000

  30/3852 [..............................]  30/3852 [..............................] - ETA: 10:09 - loss: 1.9865 - acc: 0.3000

  40/3852 [..............................]  40/3852 [..............................] - ETA: 9:36 - loss: 2.0641 - acc: 0.2500 

...

 350/3852 [=>............................] 350/3852 [=>............................] - ETA: 7:28 - loss: 2.1059 - acc: 0.2657

 360/3852 [=>............................] 360/3852 [=>............................] - ETA: 7:26 - loss: 2.1095 - acc: 0.2611

 370/3852 [=>............................] 370/3852 [=>............................] - ETA: 7:24 - loss: 2.1138 - acc: 0.2568

 380/3852 [=>............................] 380/3852 [=>............................] - ETA: 7:21 - loss: 2.1135 - acc: 0.2579

 390/3852 [==>...........................] 390/3852 [==>...........................] - ETA: 7:20 - loss: 2.1166 - acc: 0.2564

In [28]:
C = 10
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print("Expected: {}, Prediction: {}".format(Y_test[i], num))

Expected: 9, Prediction: 1
Expected: 9, Prediction: 1
Expected: 9, Prediction: 1
Expected: 8, Prediction: 1
Expected: 9, Prediction: 1
Expected: 7, Prediction: 1
Expected: 6, Prediction: 1
Expected: 8, Prediction: 1
Expected: 9, Prediction: 1
Expected: 8, Prediction: 1
Expected: 4, Prediction: 1
Expected: 5, Prediction: 1
Expected: 5, Prediction: 1
Expected: 5, Prediction: 1
Expected: 8, Prediction: 1
Expected: 7, Prediction: 1
Expected: 7, Prediction: 1
Expected: 5, Prediction: 1
Expected: 6, Prediction: 1
Expected: 4, Prediction: 1
Expected: 4, Prediction: 1
Expected: 4, Prediction: 1
Expected: 4, Prediction: 1
Expected: 5, Prediction: 1
Expected: 4, Prediction: 1
Expected: 4, Prediction: 1
Expected: 4, Prediction: 1
Expected: 0, Prediction: 1
Expected: 9, Prediction: 1
Expected: 8, Prediction: 1
Expected: 6, Prediction: 1
Expected: 7, Prediction: 1
Expected: 8, Prediction: 1
Expected: 5, Prediction: 1
Expected: 5, Prediction: 1
Expected: 4, Prediction: 1
Expected: 4, Prediction: 1
E